In [1]:
# Importing Dependencies
import pandas as pd


In [2]:
# Importing Files
steam = "Original Files/steam.csv"
steam_media_data = "Original Files/steam_media_data.csv"
steam_support_info = "Original Files/steam_support_info.csv"
steam_tag = "Original Files/steamspy_tag_data.csv"

Games = pd.read_csv(steam)
steam_media_data_df = pd.read_csv(steam_media_data)
steam_support_info_df = pd.read_csv(steam_support_info)
steam_tag_df = pd.read_csv(steam_tag)

In [3]:
# Shape of the DF
Games.shape

(27075, 18)

In [4]:
# Games DF
Games.to_csv('Files/Games.csv')

In [5]:
Games.head(1000)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65780,ARMA: Gold Edition,2011-07-13,1,Bohemia Interactive,Bohemia Interactive,windows,16,Single-player;Multi-player,Action;Simulation;Strategy,Simulation;Action;Military,0,548,209,22,22,500000-1000000,5.99
996,65790,ARMA: Cold War Assault,2011-08-08,1,Bohemia Interactive,Bohemia Interactive,windows,16,Single-player,Action;Simulation;Strategy,Simulation;Action;Strategy,0,3195,942,28,39,1000000-2000000,3.49
997,65800,Dungeon Defenders,2011-10-18,1,Trendy Entertainment,Trendy Entertainment,windows;mac;linux,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Indie;RPG;Strategy,Tower Defense;RPG;Co-op,114,10693,932,1993,1205,1000000-2000000,9.99
998,65930,The Bureau: XCOM Declassified,2013-08-22,1,2K Marin,2K,windows;mac,16,Single-player;Steam Achievements;Full controll...,Action,Action;Tactical;Sci-fi,41,6661,2923,201,168,2000000-5000000,14.99


In [6]:
# Languages DF
languages_df = pd.DataFrame(Games["english"]).drop_duplicates().reset_index(drop=True)
languages_df.columns = ['language']
languages_df = languages_df.replace([1,0],["english","other"])
languages_df.to_csv('Files/Languages.csv')
languages_df.head()

,language
0,english
1,other


In [7]:
# Developers DF
developers_df = pd.DataFrame(Games["developer"]).drop_duplicates().reset_index(drop=True)
developers_df.to_csv('Files/Developers.csv')
developers_df.head()

,developer
0,Valve
1,Gearbox Software
2,Valve;Hidden Path Entertainment
3,Mark Healey
4,Tripwire Interactive


In [8]:
# Publishers DF
publishers_df = pd.DataFrame(Games["publisher"]).drop_duplicates().reset_index(drop=True)
publishers_df.to_csv('Files/Publishers.csv')
publishers_df.head()

,publisher
0,Valve
1,Mark Healey
2,Tripwire Interactive
3,Ritual Entertainment
4,Introversion Software


In [9]:
# Owners DF
owners_df = pd.DataFrame(Games["owners"]).drop_duplicates().reset_index(drop=True)
owners_df.to_csv('Files/Publishers.csv')
owners_df.head()

,owners
0,10000000-20000000
1,5000000-10000000
2,2000000-5000000
3,20000000-50000000
4,100000000-200000000


In [10]:
# Categories DF
categories_df = Games["categories"]
# Splitting the text separated by columns
categories_df = categories_df.str.split(";",expand=True)
categories_list = []
# Appending all the cells in a list
for col in range(len(categories_df.columns)):
    for row in range(len(categories_df)):
        categories_list.append(categories_df.iloc[row][col])
        


In [11]:
# Converting the list to dataframe, dropping duplicates and resetting index
categories_list_df = pd.DataFrame(categories_list).drop_duplicates().reset_index(drop = True)
# Renaming column
categories_list_df.columns = ["category"]
# Dropping entry 24 which is the "None" value
categories_list_df = categories_list_df.drop(24).reset_index(drop=True)
categories_list_df.to_csv("Files/categories.csv")
categories_list_df

,category
0,Multi-player
1,Single-player
2,Steam Workshop
3,Partial Controller Support
4,Online Multi-Player
5,Steam Cloud
6,Steam Leaderboards
7,MMO
8,Steam Trading Cards
9,Steam Achievements


In [12]:
# Platforms DF
platforms_df = Games["platforms"]
# Splitting the text separated by columns
platforms_df = platforms_df.str.split(";",expand=True)
platforms_list = []
# Appending all the cells in a list
for col in range(len(platforms_df.columns)):
    for row in range(len(platforms_df)):
        platforms_list.append(platforms_df.iloc[row][col])
        


In [13]:
# Converting the list to dataframe, dropping duplicates and resetting index
platforms_df = pd.DataFrame(platforms_list).drop_duplicates().reset_index(drop = True)
# Renaming column
platforms_df.columns = ["platform"]
# Dropping entry 24 which is the "None" value
platforms_df = platforms_df.drop(3).reset_index(drop=True)
platforms_df.to_csv("Files/platforms.csv")
platforms_df

,platform
0,windows
1,mac
2,linux


In [14]:
# Tags DF
tags_df = Games["steamspy_tags"]
# Splitting the text separated by columns
tags_df = tags_df.str.split(";",expand=True)
tags_list = []
# Appending all the cells in a list
for col in range(len(tags_df.columns)):
    for row in range(len(tags_df)):
        tags_list.append(tags_df.iloc[row][col])

In [15]:
# Converting the list to dataframe, dropping duplicates and resetting index
tags_df = pd.DataFrame(tags_list).drop_duplicates().reset_index(drop = True)
# Renaming column
tags_df.columns = ["platform"]
# Dropping entry 24 which is the "None" value
tags_df.to_csv("Files/tags.csv")
tags_df

,platform
0,Action
1,FPS
2,Puzzle
3,Free to Play
4,Zombies
...,...
335,Faith
336,2.5D
337,Blood
338,Stylized


In [16]:
# Genres DF
genres_df = Games["genres"]
# Splitting the text separated by columns
genres_df = genres_df.str.split(";",expand=True)
genres_list = []
# Appending all the cells in a list
for col in range(len(genres_df.columns)):
    for row in range(len(genres_df)):
        genres_list.append(genres_df.iloc[row][col])

In [17]:
# Converting the list to dataframe, dropping duplicates and resetting index
genres_df = pd.DataFrame(genres_list).drop_duplicates().reset_index(drop = True)
# Renaming column
genres_df.columns = ["platform"]
# Dropping entry 24 which is the "None" value
genres_df.to_csv("Files/genres.csv")
genres_df

,platform
0,Action
1,Indie
2,Strategy
3,RPG
4,Animation & Modeling
5,Casual
6,Simulation
7,Racing
8,Adventure
9,Violent


In [18]:
# Game_Category
game_cat = Games[["appid","categories"]]
game_cat["categories"] = game_cat["categories"].str.split(";|,")
game_cat = game_cat.explode("categories").reset_index(drop=True)
game_cat.to_csv("Files/game_category.csv")
game_cat

<ipython-input-18-5ed274d20e5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_cat["categories"] = game_cat["categories"].str.split(";|,")


,appid,categories
0,10,Multi-player
1,10,Online Multi-Player
2,10,Local Multi-Player
3,10,Valve Anti-Cheat enabled
4,20,Multi-player
...,...,...
89721,1065650,Steam Cloud
89722,1066700,Single-player
89723,1066700,Steam Cloud
89724,1069460,Single-player


In [19]:
# Game_platform
game_plat = Games[["appid","platforms"]]
game_plat["platforms"] = game_plat["platforms"].str.split(";|,")
game_plat = game_plat.explode("platforms").reset_index(drop=True)
game_plat.to_csv("Files/game_platform.csv")
game_plat

<ipython-input-19-a129c8e72ca0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_plat["platforms"] = game_plat["platforms"].str.split(";|,")


,appid,platforms
0,10,windows
1,10,mac
2,10,linux
3,20,windows
4,20,mac
...,...,...
40366,1065650,windows
40367,1066700,windows
40368,1066700,mac
40369,1069460,windows


In [20]:
# Game_tag
game_tag = Games[["appid","steamspy_tags"]]
game_tag["steamspy_tags"] = game_tag["steamspy_tags"].str.split(";|,")
game_tag = game_tag.explode("steamspy_tags").reset_index(drop=True)
game_tag.to_csv("Files/game_tag.csv")
game_tag

<ipython-input-20-6cd5d231c41c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_tag["steamspy_tags"] = game_tag["steamspy_tags"].str.split(";|,")


,appid,steamspy_tags
0,10,Action
1,10,FPS
2,10,Multiplayer
3,20,Action
4,20,FPS
...,...,...
77997,1066700,Casual
77998,1066700,Adventure
77999,1069460,Indie
78000,1069460,Casual


In [21]:
# game_genre
game_genre = Games[["appid","genres"]]
game_genre["genres"] = game_genre["genres"].str.split(";|,")
game_genre = game_genre.explode("genres").reset_index(drop=True)
game_genre.to_csv("Files/game_genre.csv")
game_genre

<ipython-input-21-3a26a9d2e91c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_genre["genres"] = game_genre["genres"].str.split(";|,")


,appid,genres
0,10,Action
1,20,Action
2,30,Action
3,40,Action
4,50,Action
...,...,...
76457,1066700,Casual
76458,1066700,Indie
76459,1069460,Adventure
76460,1069460,Casual
